# imports
from __future__ import print_function

#Week 1

## Lecture 2: Decorators in Python (continued), Debugging Tools, Pytest
## Breakouts: Adding a Timer to a Function Using a Decorator, Using ipdb.set_trace()
## Lab: Pytest example; for new students: Brief introduction to Python scientific computation suite, gedit, basic command line commands.

## I. \*args and \*\*kwargs.

In [1]:
# First *args...
# *args allows you to use an arbitrary number of positional arguments when calling a function
def new_fun(*args):
    if len(args) > 0:
        print('Arguments received in new_fun: {}'.format(args))
        sum = 0
        for i in range(len(args)):
            sum += args[i] 
        return sum
    else:
        raise Exception('No numbers to sum.')

print('sum = {}'.format(new_fun(4, 2)))
print('sum = {}'.format(new_fun(4, 2, 5, 6, 2, 5))) 
# print('sum = {}'.format(new_fun()))


Arguments received in new_fun: (4, 2)
sum = 6
Arguments received in new_fun: (4, 2, 5, 6, 2, 5)
sum = 24


In [2]:
# Next **kwargs...
# **kwargs allows you to use an arbitrary number of keyword arguments when calling a function
def new_fun2(**kwargs):
    if len(kwargs)>0:
        for key, value in kwargs.iteritems():
            print('key and value of kwargs: {}, {}'.format(key, value))
        return 

new_fun2(name = 'David', weight = 200)
print('Adding another kwarg:')
new_fun2(name = 'David', weight = 200, home_town = 'San Francisco')

# Note: the keyword arguments are treated by python as a dictionary.

key and value of kwargs: name, David
key and value of kwargs: weight, 200
Adding another kwarg:
key and value of kwargs: name, David
key and value of kwargs: weight, 200
key and value of kwargs: home_town, San Francisco


## Why is this useful?
## Sometimes you don't know how many arguments an inner function takes.  Using \*args and \*\*kwargs gives you that flexibility...As you will see shortly.
    

## II. Decorators

## Consider the following problem:

In [3]:
# first define a simple function
def hello_func():
    return "hello world"
print(hello_func())

hello world


In [4]:
# now consider this
def outer(fun):
    def inner():
        return fun
    return inner
    
foo = outer(hello_func)

In [7]:
print(foo()())

hello world
<function hello_func at 0x102e46b90>


## Question: Using foo, how would you get the python shell to print "hello world"?

In [8]:
'''
    Making hello_func a little more useful:
'''
def outer(func):
    def inner(city_name):
        return func(city_name)
    return inner

def hello_func(strg):
    print('hello ' + strg)

    
foo = outer(hello_func)
foo('San Francisco')
foo('Oakland')
foo('World')
# To the object hello_func, which is the result of another function passing through the function count only once
#  Thus to this oject inner.counter will always be 0


hello San Francisco
hello Oakland
hello World


In [9]:
'''
    Making things a little more abstract by using *args.
    At this point, there is no advantage gained, just another way of doing things.
    I also called foo to hello_fun.

'''
def outer(func):
    def inner(*args):
        return func(*args)
    return inner

def hello_func(strg):
    print('hello ' + strg)

    
hello_fun = outer(hello_func)
hello_fun('San Francisco')
hello_fun('Oakland')
hello_fun('World')


hello San Francisco
hello Oakland
hello World


In [11]:
'''
    Let's add an attribute: count.
    I have changed the name outer to count.
'''
def count(func):
    def inner(*args):
        inner.counter += 1
        return func(*args)

    # w/o this, you can't do inner.counter += 1 in inner.
    inner.counter = 0
    
    return inner

def hello_func(strg):
    print('hello ' + strg)
    
# inner.counter = 0 is executed here, by the call to count().
hello_fun = count(hello_func)
    
# once the object hello_fun is created, and since it's the same as inner, 
# every time it's called count increases by 1.  
hello_fun('San Francisco')
hello_fun('Oakland')
hello_fun('World')
hello_fun.counter

hello San Francisco
hello Oakland
hello World


3

In [ ]:
# try a couple more times.
hello_fun('Orlando')
hello_fun('Atlanta')
hello_fun.counter

In [12]:
'''
    Would it really bother you if I changed hello_fun to hello_func?
    Hint: It shouldn't!!
'''
def count(func):
    def inner(*args):
        inner.counter += 1
        return func(*args)

    # w/o this, you can't do inner.counter += 1 in inner.
    inner.counter = 0
    
    return inner

def hello_func(strg):
    print('hello ' + strg)
    
hello_func = count(hello_func)
    
hello_func('San Francisco')
hello_func('Oakland')
hello_func('World')
hello_func.counter

hello San Francisco
hello Oakland
hello World


3

## What's really nice about this syntactic gymnastics is that
## - hello_func can do exactly what it used to do before it's passed through the function count.
## - But now it's been given an additional attribute, counter, which allows the number of function calls to be recorded!
## - In case you think this is easy, it's not!  I saw a Python guru got it (subtly) wrong.

## The Decorator as "Syntactic Suger"

In [13]:
@count
def hello_func(strg):
    print('hello ' + strg)
    
hello_func('San Francisco')
hello_func('Oakland')
hello_func('World')
hello_func('Brazil')
hello_func.counter

hello San Francisco
hello Oakland
hello World
hello Brazil


4

## Why we bothered to use \*args

In [14]:
@count
def new_fun(*args):
    if len(args) > 0:
        print('Arguments received in new_fun: {}'.format(args))
        sum = 0
        for i in range(len(args)):
            sum += args[i] 
        return sum
    else:
        raise Exception('No numbers to sum.')

print('sum = {}'.format(new_fun(4, 2)))
print('sum = {}'.format(new_fun(4, 2, 5, 6, 2, 5))) 
print(new_fun.counter)

Arguments received in new_fun: (4, 2)
sum = 6
Arguments received in new_fun: (4, 2, 5, 6, 2, 5)
sum = 24
2


## Breakout Exercise:
## 1. Write an "outer function" (sometimes called a wrapper) that times how long it takes to run a function.  It should add an attribute delta_time to the function that is passed to it.
## 2. Pass a function you would like to be timed through this outer function, and show that by printing the attribute delta_time, you can print how much time it takes to run this function.
## 3. Do the same with the decorator.

In [16]:
from time import time
def timing(func):
    def inner(*args):
        start = time()
        stuff = func(*args)
        inner.delta_time = time() - start
        return stuff

    # w/o this, you can't do inner.counter += 1 in inner.
    
    return inner



hello San Francisco
hello Oakland
hello World


2.288818359375e-05

In [17]:
@timing
def hello_func(strg):
    print('hello ' + strg)
    
hello_func = timing(hello_func)
    
hello_func('San Francisco')
hello_func('Oakland')
hello_func('World')
hello_func.delta_time

hello San Francisco
hello Oakland
hello World


2.288818359375e-05

## Decorators in Class -- an example: property

## Breakout Exercise

-  ## Write a class Bears that needs to be given the argument age at the time of instantiation
-  ## This class should have one method: 
    -  ### current_age: increases the age by 1 each time it's invoked. 
-  ## Create an instance, yogi, of the class Bears with a reasonable age.

In [34]:
class Bears:
    def __init__(self, age):
        self.age = age
        
    def current_age(self):
        self.age += 1
        
yogi = Bears(13)
print(yogi.age)
yogi.current_age()
print(yogi.age)
yogi.current_age()
print(yogi.age)

13
14
15


## Introduce a Python internal function: property;
## just as print, range, len are internal functions

In [ ]:
print(yogi.current_age)

In [ ]:
print(yogi.current_age())

In [35]:
class Bears:
    def __init__(self, age):
        self.age = age
        
    def current_age(self):
        self.age += 1
        return self.age
    
    # Turn current_age into a "property" (like an attribute)
    current_age = property(current_age)
        
yogi = Bears(0)
print(yogi.age)    

0


In [43]:
print(yogi.current_age)

8


In [ ]:
# This is no longer OK.
print(yogi.current_age())

## A more succinct way, the decorator: @property

In [845]:
class Bears:
    def __init__(self, age):
        self.age = age
        
    @property 
    def current_age(self):
        self.age += 1
        return self.age
            
yogi = Bears(0)
print(yogi.age)    

0


In [945]:
print(yogi.current_age)

100


# 5 min break

## III. Debugging Tools (pdb and ipdb)

### - pdb: Python DeBugger
### - ipdb: the IPython counterpart -- behaves just like pdb in the notebook environment, but much more interactive at the command line.  Reommended!!

### (A friendly tutorial: https://pythonconquerstheuniverse.wordpress.com/2009/09/10/debugging-in-python/)

## pdb or ipdb

- ###   set_trace() -- place it where you would want the execution to stop.
-  ###   When the program stops execution when it encounters, you will see the pdb prompt:

    pdb>
    
   ###     or

    ipdb>


- ###   At the pdb or ipdb prompt, use the following commands to find the problem (the "bug")  
   ###     1.    n -- next step.
   ###     2.   Hitting the enter/return key -- repeats the last command at the pdb prompt.
   ###     3.   p -- to print anything you want.
   ###     4.   s -- to step into a subroutine (typically, a function or a class definition).
   ###     5.   r -- if you are inside a long subroutine and you realize the problem is not in this subroutine and would like to get out immediately, use r, for return.
   ###     6.   unt (until) -- if you are stuck in a for loop that repeats, say, 1000 times, you can step all the way to the last statement of the loop and then use 

    ipdb>unt
    
    ###     you will be taken out of the for-loop.  Because unt will take you to the next statement that has a *larger* line number.  

    ###     To clarify: Unless you are at the last statement of a for/while loop, unt does exactly the same thing as n -- it takes you to the next statement.
    
   ###  7. If you lost track of where you are in the program, use l (list), the 10 lines around the one that python is about to execute will be shown.
   
   ###  8.  If you forgot "how you got here" (let's say you are in a 3-level deep function call), use w -- this shows which function is called by which function, and that function is in turn called by which function, etc. (or, "stack trace").
   
   ###  9. In the debugger, you can change the value of a variable.
   
   ###  10. If you believe you have corrected the problem (see 9. above) and would like to let the program run on its own course to the end, enter c, for continue:
   
    ipdb> c
     
   ###  11.  If you think you have found the problem and would like to quit the bebugger (in order to correct your program, and then re-run it), use q:
   
    ipdb> q
    
   ###  12. If you use pdb/ipdb in ipython notebook:
     * ###    Use it inside a function, but not at the "main" level. 
     * ###    Always terminate the pdb/ipdb (by using c or q) first before you execute another cell.  Otherwise, the notebook may be stuck in an infinite loop.
    
    
   ### In my experience, pdb/ipdb is much better than printing statements -- as it allows you to investigate multiple sources for the bug with one stop, and allows you to see clearly the logical flow of the program, and therefore makes it easier to find where the problem is. 

## A debugging example:

In [ ]:
from ipdb import set_trace

def some_analysis(nums):
    b = []
    for num in nums:
        b.append(1/(num - 5.))
    return b
                 
def some_calc(nums):
    
    # This sets a break point.
    set_trace()
    
    N = len(nums)
    
    i = 0
    total = 0    
    b = some_analysis(nums)
    while i < N:
        total += nums[i]
        i += 1 
    return float(total) / float(len(N))


a_list = [1, 2, 3, 4, 5, 6, 10]  

print(some_calc(a_list))
print('DONE!')

> <ipython-input-4-2d2057b6f91d>(14)some_calc()
     13 
---> 14     N = len(nums)
     15 

ipdb> n
> <ipython-input-4-2d2057b6f91d>(16)some_calc()
     15 
---> 16     i = 0
     17     total = 0

ipdb> 
> <ipython-input-4-2d2057b6f91d>(17)some_calc()
     16     i = 0
---> 17     total = 0
     18     b = some_analysis(nums)

ipdb> 
> <ipython-input-4-2d2057b6f91d>(18)some_calc()
     17     total = 0
---> 18     b = some_analysis(nums)
     19     while i < N:

ipdb> s
--Call--
> <ipython-input-4-2d2057b6f91d>(3)some_analysis()
      2 
----> 3 def some_analysis(nums):
      4     b = []

ipdb> 
> <ipython-input-4-2d2057b6f91d>(4)some_analysis()
      3 def some_analysis(nums):
----> 4     b = []
      5     for num in nums:

ipdb> 
> <ipython-input-4-2d2057b6f91d>(5)some_analysis()
      4     b = []
----> 5     for num in nums:
      6         b.append(1/(num - 5.))

ipdb> p b
[]
ipdb> 
[]
ipdb> 
[]
ipdb> n
> <ipython-input-4-2d2057b6f91d>(6)some_analysis()
      5     for num in

## Breakout Exercise: Let's fix it!
## ... or did we??
## Your turn to use set_trace to find the problem.
- ## Turn on line number with Escape-(lower case)L.
- ## Put set_trace at the same line as before.  
- ## Even if you have spotted the problem (*shhh...*), practice stepping through all the lines of the program at least once, by using ipdb commands.
- ## Demonstrate to me, using ipdb, exactly where the problem is.

In [5]:
from ipdb import set_trace

def some_analysis(nums):
    b = []
    for num in nums:
        b.append(1/(num - 5.))
    return b
                 
def some_calc(nums):
    
    # This sets a break point.
    set_trace()
    
    N = len(nums)
    
    i = 0
    total = 0    
    b = some_analysis(nums)
    while i < N:
        total += nums[i]
        i += 1 
    return float(total) / float(len(N))


#a_list = [1, 2, 3, 4, 5, 6, 10]  

epsilon = 1e-6
a_list = [1, 2, 3, 4, 5+epsilon, 6, 10]  

print(some_calc(a_list))
print('DONE!')


> <ipython-input-5-e08b7d5d8b07>(14)some_calc()
     13 
---> 14     N = len(nums)
     15 

ipdb> n
> <ipython-input-5-e08b7d5d8b07>(16)some_calc()
     15 
---> 16     i = 0
     17     total = 0

ipdb> n
> <ipython-input-5-e08b7d5d8b07>(17)some_calc()
     16     i = 0
---> 17     total = 0
     18     b = some_analysis(nums)

ipdb> n
> <ipython-input-5-e08b7d5d8b07>(18)some_calc()
     17     total = 0
---> 18     b = some_analysis(nums)
     19     while i < N:

ipdb> n
> <ipython-input-5-e08b7d5d8b07>(19)some_calc()
     18     b = some_analysis(nums)
---> 19     while i < N:
     20         total += nums[i]

ipdb> n
> <ipython-input-5-e08b7d5d8b07>(20)some_calc()
     19     while i < N:
---> 20         total += nums[i]
     21         i += 1

ipdb> l
     15 
     16     i = 0
     17     total = 0
     18     b = some_analysis(nums)
     19     while i < N:
---> 20         total += nums[i]
     21         i += 1
     22     return float(total) / float(N)
     23 
     24 
    

## VI. Testing

In [1]:
# The assert statement
a = 2
assert a == 2 

In [2]:
a = 2
assert a == 3 

AssertionError: 

## Pytest -- A Very Brief Introduction

## http://pytest.org/latest/getting-started.html

## Installation:

    > pip install -U pytest
    
## The "switch" -U: update pytest if you already have it.

## To use pytest:

- ## Write the tests in a function that starts with test\_
- ## You can then use pytest on your program like this

        > py.test your_awesome_program.py
        
   ## pytest will collect all functions that start with test\_ and run them
    
## See example: intro_pytest.py

## Breakout Exercise:

- ## Write a program called falling_obj.py
- ## Use g = 9.8
- ## It should have two functions:
    - ## height(h0, v0, t), which takes three arguments, the initial height h0, the initial velocity v0, and time, t, and returns the height of a falling object at time t.  If the result is negative, return 0.
    - ## vel(h0, v0, t), which takes the same three arguments, and returns the velocity at time t.  It should check the height of the object at t by calling the function height.  If the returned value of height is negative, then return 0 as the velocity.
- ## It should have two test functions:
    - ## test_height
    - ## test_vel
##   They should, respectively, check whether the functions height and vel return the correct values for a set of specific values of (h0, v0, t), say, (10, 2, 1).  Obviously, *you* should calculate the answers for each function by hand yourself.  Check also for very large t, say 100000 sec, both functions should return 0.  If everything works as you expect, then intentionally change the functions height and vel, one at a time, so that the formula is a little wrong -- and see if pytest catches the error.


## End of Week1-2